In [105]:
open System.IO

let input = File.ReadAllLines("./input.txt")

let charToIntArray = Array.map (fun s -> (s.ToString() |> int))
let grid = 
    input
    |> Array.map (fun s -> s.ToCharArray())
    |> Array.map charToIntArray

let xs = [1..grid.Length-2]
let ys = [1..grid[0].Length-2]

In [106]:
let visibleUp (xpos:int, ypos:int) = 
    [|0..xpos-1|]
    |> Array.map (fun x -> grid[xpos][ypos] > grid[x][ypos])
    |> Array.reduce (fun (a:bool) (b:bool) -> a && b)

let visibleDown (xpos:int, ypos:int) = 
    [|xpos+1..grid.Length-1|] 
    |> Array.map (fun x -> grid[xpos][ypos] > grid[x][ypos])
    |> Array.reduce (fun (a:bool) (b:bool) -> a && b)

let visibleLeft (xpos:int, ypos:int) = 
    [|0..ypos-1|] 
    |> Array.map (fun y -> grid[xpos][ypos] > grid[xpos][y])
    |> Array.reduce (fun (a:bool) (b:bool) -> a && b)

let visibleRight (xpos:int, ypos:int) = 
    [|ypos+1..grid[0].Length-1|] 
    |> Array.map (fun y -> grid[xpos][ypos] > grid[xpos][y])
    |> Array.reduce (fun (a:bool) (b:bool) -> a && b)

let visibleAny (xpos:int, ypos: int) = 
    visibleUp (xpos, ypos) 
 || visibleDown (xpos, ypos) 
 || visibleLeft (xpos, ypos) 
 || visibleRight (xpos, ypos)


In [107]:
// part one
let perimiterTotal = (grid.Length * 2) + (grid[0].Length * 2) - 4

let innerTotal = 
    xs 
    |> List.collect (fun x -> ys |> List.map (fun y -> x, y))
    |> List.map visibleAny
    |> List.sumBy (fun b -> if b then 1 else 0)

perimiterTotal + innerTotal

1560

In [108]:
let takeUntil predicate (s:seq<_>) = 
  /// Iterates over the enumerator, yielding elements and
  /// stops after an element for which the predicate does not hold
  let rec loop (en:IEnumerator<_>) = seq {
    if en.MoveNext() then
      // Always yield the current, stop if predicate does not hold
      yield en.Current
      if predicate en.Current then
        yield! loop en }

  // Get enumerator of the sequence and yield all results
  // (making sure that the enumerator gets disposed)
  seq { use en = s.GetEnumerator()
        yield! loop en }

let scenicUp (xpos:int, ypos:int) = 
    [|xpos-1..-1..0|] 
    |> Seq.map (fun x -> grid[xpos][ypos] > grid[x][ypos])
    |> takeUntil (fun x -> x = true)
    |> Seq.length

let scenicDown (xpos:int, ypos:int) = 
    [|xpos+1..grid.Length-1|] 
    |> Seq.map (fun x -> grid[xpos][ypos] > grid[x][ypos])
    |> takeUntil (fun x -> x = true)
    |> Seq.length

let scenicLeft (xpos:int, ypos:int) = 
    [|ypos-1..-1..0|] 
    |> Seq.map (fun y -> grid[xpos][ypos] > grid[xpos][y])
    |> takeUntil (fun x -> x = true)
    |> Seq.length

let scenicRight (xpos:int, ypos:int) = 
    [|ypos+1..grid[0].Length-1|] 
    |> Seq.map (fun y -> grid[xpos][ypos] > grid[xpos][y])
    |> takeUntil (fun x -> x = true)
    |> Seq.length
    
let scenicScore (xpos:int, ypos:int) = 
    [|scenicUp (xpos, ypos); scenicDown (xpos, ypos); scenicLeft (xpos, ypos); scenicRight (xpos, ypos)|]
    |> Array.fold (fun acc x -> if x = 0 then (acc * 1) else (acc * x)) 1 

xs 
    |> List.collect (fun x -> ys |> List.map (fun y -> x, y))
    |> List.map scenicScore
    |> List.max

252000